config.py:



In [ ]:
import os

# model configs
NUM_SAMPLES = 3000  # not a total number of samples, but max number of samples with the same 'ship_count'
VALIDATION_SET_SIZE = 0.1 # number from 0 to 1.0
GAUSSIAN_NOISE = 0.1
NB_EPOCHS = 40
BATCH_SIZE = 64
PATCH_SIZE = 256
INPUT_DATA_DIM = (PATCH_SIZE, PATCH_SIZE, 3)

# env_configs
BASE_DIR = '/kaggle/input/airbus-ship-detection'
TEST_IMG_DIR = os.path.join(BASE_DIR,'test_v2')
TRAIN_IMG_DIR = os.path.join(BASE_DIR,'train_v2')
TRAIN_DATASET_CSV = os.path.join(BASE_DIR,'train_ship_segmentations_v2.csv')

# WEIGHTS_DIR = 'weights'
# WEIGHTS_FILE = 'model.{epoch:02d}-{val_loss:.2f}.weights.h5'
# WEIGHTS_PATH = os.path.join(WEIGHTS_DIR, WEIGHTS_FILE)

MODEL_DIR = 'model'
MODEL_FILE = 'model.{epoch:02d}-{val_loss:.2f}.keras'
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_FILE)

# model testing configs
MODEL_TO_TEST = '/kaggle/input/10h-model/keras/10h/1/model.30-0.35.keras'
# MODEL_TO_TEST_PATH = os.path.join(MODEL_DIR, MODEL_TO_TEST)
MODEL_TO_TEST_PATH = os.path.join(MODEL_TO_TEST)

utils.py:

In [ ]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from skimage.transform import resize
import matplotlib.pyplot as plt

#utility functions
def rle_to_mask(starts, lengths, height, width):
    # Create an empty array of zeros of shape (height, width)
    mask = np.zeros(height * width, dtype=np.uint8)

    # For each start and length, set the corresponding values in the mask to 1
    for start, length in zip(starts, lengths):
        mask[start:start + length] = 1

    # Reshape the mask into the desired dimensions
    mask = mask.reshape((height, width))
    mask = mask.T
    return mask

def create_mask(mask_array, width=768, height=768):
    masks = np.zeros((width, height), dtype=np.int16)
    # if element == element:
    if isinstance(mask_array, str):
        split = mask_array.split()
        startP, lengthP = [np.array(x, dtype=int) for x in (split[::2], split[1::2])]
        masks += (rle_to_mask(startP, lengthP, width, height))
    return masks

def generate_prediction(model, img_dir, img_name):
    img = os.path.join(img_dir, img_name)
    img = Image.open(img)
    img = np.array(img)
    img = resize(img, (PATCH_SIZE, PATCH_SIZE), anti_aliasing=True)
    img = tf.expand_dims(img, axis=0)
    pred = model.predict(img)
    print(f"prediction shape - {pred.shape}")
    return pred, img

def visualise_prediction(model, img_dir, img_name):
    pred, img = generate_prediction(model, img_dir, img_name)
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(img[0])
    plt.title("Original Image")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(pred[0])
    plt.title("Predicted Mask")
    plt.axis("off")
    plt.show()

losses.py:


In [ ]:
import tensorflow.keras.backend as K

# loss functions
def dice_score(y_true, y_pred, smooth=1e-6):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3]  )
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)


def dice_loss(y_true, y_pred):
    return 1 - dice_score(y_true, y_pred)

def bce_loss(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=[1, 2, 3])

def dice_bce_loss(y_true, y_pred):
    dice_loss_value = dice_loss(y_true, y_pred)
    bce_loss_value = bce_loss(y_true, y_pred)
    return dice_loss_value + bce_loss_value

In [ ]:
from tensorflow import keras
import random


# Handpicked images for nice results :)
handpicked_test_imgs = ['ba6fe8a25.jpg','8dd7d6368.jpg','534e10e90.jpg','711be1fce.jpg','8e6b39bb1.jpg','45d43380c.jpg','2868e5640.jpg','a00886458.jpg','0c2848844.jpg','d9b95022a.jpg','ba9c3d11a.jpg','3566fb758.jpg','abb672b82.jpg','acb7dd8d2.jpg', '30e126c21.jpg']

# choose 100 random images from test directory for unbiased results
file_names = os.listdir(TEST_IMG_DIR)
test_imgs = random.sample(file_names, 33)

model = keras.models.load_model(MODEL_TO_TEST_PATH, custom_objects={'dice_bce_loss': dice_bce_loss, "dice_score": dice_score})

for i in handpicked_test_imgs:
   visualise_prediction(model, TEST_IMG_DIR, i) 

for i in test_imgs:
   visualise_prediction(model, TEST_IMG_DIR, i) 